一旦仮で作ったところまで

In [1]:
import pandas as pd

# データAとデータBのCSVファイルのパス
data_a_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/original_data/race_table/combined.csv'  # データAのCSVファイルパス
data_b_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/merged_data/merged_sorted.csv'  # データBのCSVファイルパス

# CSVファイルからデータを読み込む
data_a = pd.read_csv(data_a_path,low_memory=False)
data_b = pd.read_csv(data_b_path)
data_a=data_a.dropna()
data_b=data_b.dropna()
data_a=data_a.drop(columns=['first_horses', 'second_horses', 'third_horses', 'Time', 'Jockey', 'Trainer'])
# Code列でデータを結合 (data_bにdata_aの情報を追加)
merged_data = pd.merge(data_b, data_a, on='Code', how='left')

#結果の確認
merged_data.to_csv('../notebook/data-shin/merged_data_final.csv', index=False)
# 各列のユニークな値とその出現回数を取得
for i in merged_data.columns:
    value_counts = merged_data[i].value_counts()  # 各値とそのカウント
    print(f"{i}: {value_counts}")




KeyboardInterrupt: 

なんかパスが奇妙なエラーを吐くので一旦絶対パス使ってます。

In [1]:
import matplotlib.pyplot as plt
print("matplotlib is successfully installed!")


matplotlib is successfully installed!


In [3]:
import pandas as pd
import numpy as np

# データの読み込み（エンコーディングを指定して文字化け防止）
file_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/notebook/data-shin/merged_data_final.csv'
df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)

# 1. Sex/Ageを分割して新しい列に追加
df[['Sex', 'Age']] = df['Sex/Age'].str.extract(r'([A-Za-z]+)(\d+)')
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')  # 数値型に変換、変換できないものはNaNに
df.drop(columns=['Sex/Age'], inplace=True)  # 元のSex/Age列を削除

# 2. カテゴリカルデータをOne-Hot Encoding
categorical_columns = ['Sex', 'Jockey', 'Track', 'Weather', 'Condition', 'Trainer', 'Banushi', 'Horse Name']
df = pd.get_dummies(df, columns=categorical_columns, dtype=np.uint8)  # メモリ効率の良い型を指定

# 3. 欠損値の処理（欠損値を削除、文字列データに対してNaNを取り扱う）
df.dropna(subset=['Sex', 'Age'], inplace=True)  # SexやAgeに欠損があれば削除
df.dropna(inplace=True)  # その他の欠損値がある行を削除

# 4. インデックスの異常データを削除（インデックスが重複している行を削除）
df = df.loc[~df.index.duplicated(), :]  # 重複するインデックスを削除

# 5. メモリの最適化（データ型の変更）
# 数値型のカラムに対して適切な型に変換
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = pd.to_numeric(df[col], downcast='float')
for col in df.select_dtypes(include=['int64']).columns:
    df[col] = pd.to_numeric(df[col], downcast='integer')

# 6. 不要な列の削除（CodeやRankなど予測に不要なら削除）
columns_to_drop = ['Code', 'Rank', 'Frame Rank']  # 必要に応じて列を指定
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 7. 前処理後のデータを新しいCSVファイルに保存
output_path = 'processed_horse_data_optimized.csv'  # 保存するファイル名を指定
df.to_csv(output_path, index=False)

print(f"前処理が完了しました。処理済みデータは {output_path} に保存されました。")


: 

features = [
    "Rank","Kinryou","Nobori","Ninki",
    "Race Number","Distance","Weight","Weight Change",
    "Sex","Age","Ground_ダ","Ground_芝","Ground_障","Condition_不",
    "Condition_稍","Condition_良","Condition_重","Weather_小雨","Weather_小雪",
    "Weather_晴","Weather_曇","Weather_雨","Weather_雪"
]
target = "Time_x"

ランクは相対的であると判断し削除。人気は予想がすでに反映されている可能性がある。。。？

In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# データの読み込み
file_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/format_data.csv'
df = pd.read_csv(file_path)

# 必要な特徴量とターゲット変数の選択
X = df[[
    "Kinryou", "Nobori","Ninki",
    "Race Number", "Distance", "Weight", "Weight Change",
    "Sex", "Age", "Ground_ダ", "Ground_芝", "Ground_障", 
    "Condition_不", "Condition_稍", "Condition_良", "Condition_重", 
    "Weather_小雨", "Weather_小雪", "Weather_晴", "Weather_曇", "Weather_雨", "Weather_雪"
]]

y = df['Time_x']  # ターゲット変数

# 欠損値を含む行を削除
X = X.dropna()
y = y.loc[X.index]  # Xのインデックスに合わせてyも調整

# データの分割 (訓練データとテストデータ)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Lasso回帰モデルの作成と学習
lasso = Lasso(alpha=0.1)  # alphaは正則化の強さ
lasso.fit(X_train, y_train)

# 予測
y_pred = lasso.predict(X_test)

# 評価 (MSEを表示)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Lassoモデルの係数を表示
print("Lasso coefficients:")
print(lasso.coef_)


/var/folders/0y/s3g9vbzd3vvch0mp2zlmz2pr0000gn/T/ipykernel_2030/854844578.py:8: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Mean Squared Error: 6.639521235338716
Lasso coefficients:
[ 1.61775345e-01  2.04326772e-02  1.82504574e-01 -5.86524520e-02
 -2.39697959e-01  6.84842906e-02 -1.16137627e-02  6.31409587e-04
  0.00000000e+00  2.37265676e-01  0.00000000e+00 -6.39200144e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00]


制度は微妙だが動いてそうだから次に進めます。
以下は学習後に実際に予測させてます。（最初にモデルを作ってます。）

In [15]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

# データの読み込み
file_path = '/Users/whale217/dev/ProjectTeam5/workspace/team1/data/format_data.csv'
df = pd.read_csv(file_path)

# Rankを除外して特徴量とターゲット変数を選択
X = df[[
    "Kinryou", "Nobori", "Ninki",
    "Race Number", "Distance", "Weight", "Weight Change",
    "Sex", "Age", "Ground_ダ", "Ground_芝", "Ground_障", 
    "Condition_不", "Condition_稍", "Condition_良", "Condition_重", 
    "Weather_小雨", "Weather_小雪", "Weather_晴", "Weather_曇", "Weather_雨", "Weather_雪"
]]

y = df['Time_x']  # ターゲット変数

# 欠損値を含む行を削除
X = X.dropna()
y = y.loc[X.index]  # Xのインデックスに合わせてyも調整

# データの分割 (訓練データとテストデータ)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Lasso回帰モデルの作成と学習
lasso = Lasso(alpha=0.1)  # alphaは正則化の強さ
lasso.fit(X_train, y_train)

# 予測
y_pred = lasso.predict(X_test)

# 評価 (MSEを表示)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Lassoモデルの係数を表示
print("Lasso coefficients:")
print(lasso.coef_)

#モデルの保持
model_filename = 'lasso_model.pkl'
joblib.dump(lasso, model_filename)

/var/folders/0y/s3g9vbzd3vvch0mp2zlmz2pr0000gn/T/ipykernel_2030/81795970.py:9: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Mean Squared Error: 6.703728165706276
Lasso coefficients:
[ 2.16649614e-02  1.86280339e-01 -4.23405786e-02 -2.40494524e-01
  6.84975337e-02 -1.17463394e-02  8.73614315e-04  0.00000000e+00
  2.39945628e-01  0.00000000e+00 -6.38672882e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00]


['lasso_model.pkl']

In [14]:
# モデルの読み込み
loaded_model = joblib.load('lasso_model.pkl')

# 新しいデータに対して予測を行う
new_data = pd.DataFrame({
    "Kinryou": [55.0],
    "Nobori": [35.2],
    "Ninki": [3],
    "Race Number": [5],
    "Distance": [1800],
    "Weight": [470],
    "Weight Change": [2],
    "Sex": [1],
    "Age": [4],
    "Ground_ダ": [1],
    "Ground_芝": [0],
    "Ground_障": [0],
    "Condition_不": [0], "Condition_稍": [1], "Condition_良": [0], "Condition_重": [0],
    "Weather_小雨": [0], "Weather_小雪": [0], "Weather_晴": [1], "Weather_曇": [0],
    "Weather_雨": [0], "Weather_雪": [0]
})

# モデルで新しいデータを予測
predicted_time = loaded_model.predict(new_data)

# 予測結果を表示
print(f"予測されたTime_x: {predicted_time[0]}")


予測されたTime_x: 117.39123526056599
